In [585]:
import random
random.seed(109)

In [654]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        current_player = self.current_player
        
        if pit < 1:
            print("Invalid pit selected")
            return False
        
        if current_player == 1:
            pit_index = pit - 1
        elif current_player == 2:
            pit_index = self.pits_per_player + pit
            
        if pit_index == self.p1_mancala_index or pit_index == self.p2_mancala_index:
            print("Cannot play from a mancala")
            return False
        
        if self.board[pit_index] <= 0:
            print("Cannot play from a pit with no stones")
            return False
        
        self.moves.append((current_player, pit))
        return True
        
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        if self.current_player == 1:
            pits = range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)
        else:
            pits = range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)
            
        valid_pits = [pit for pit in pits if self.board[pit] > 0]
        
        if not valid_pits:
            print(f"Player {self.current_player} has no valid moves.")
            return None
        
        selected_pit = random.choice(valid_pits)
        print(f"Player {self.current_player} randomly selects pit {selected_pit + 1} (index {selected_pit}).")
        
        return selected_pit + 1
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return
        
        if self.winning_eval() == True:
            print("GAME OVER")
            return
        
        # Check for the current player
        current_player = self.current_player
        
        # Assign the pit_index based on the player
        if current_player == 1:
            pit_index = pit - 1
        elif current_player == 2:
            pit_index = self.pits_per_player + pit
        
        # Pick up the stones
        stones = self.board[pit_index]
        # Remove them from the selected pit
        self.board[pit_index] = 0
        
        index = pit_index
        
        # While there are still available stones
        while stones > 0:
            # Wrap around the board
            index = (index + 1) % len(self.board)
            
            # Skip the opponents mancala
            if (current_player == 1 and index == self.p2_mancala_index) or (current_player == 2 and index == self.p1_mancala_index):
                continue
                
            # Put a stone in the pit
            self.board[index] += 1
            # Take a stone from the player
            stones -= 1
            
        if current_player == 1 and self.p1_pits_index[0] <= index <= self.p1_pits_index[1] and self.board[index] == 1:
            opposite_index = 12 - index
            
            if self.board[opposite_index] > 0:
                # Print out the number of stones here
                print(f"Player 1 captures stones from Player 2 pit {opposite_index - 6}.")
                
                if self.board[index] > 8:
                    self.board[self.p1_mancala_index] += self.board[opposite_index] + 1
                    self.board[index] = 0
                    self.board[opposite_index] = 0
                
                
        elif current_player == 2 and self.p2_pits_index[0] <= index <= self.p2_pits_index[1] and self.board[index] == 1:
            opposite_index = 12 - index
            
            if self.board[opposite_index] > 0:
                # Print out the number of stones here
                print(f"Player 2 captures stones from Player 1 pit {opposite_index}.")
                
                if self.board[index] > 8:
                    self.board[self.p2_mancala_index] += self.board[opposite_index] + 1
                    self.board[index] = 0
                    self.board[opposite_index] = 0
                
        
        if (current_player == 1):
            self.current_player = 2
        else:
            self.current_player = 1
            
        print(f"Turn: Player {self.current_player}")

        
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        p1_pits_empty = all(stones == 0 for stones in self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
        p2_pits_empty = all(stones == 0 for stones in self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])
        
        if p1_pits_empty or p2_pits_empty:
            if (self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]):
                print(f"Player 1 Wins with {self.board[self.p1_mancala_index]} stones")
            elif (self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]):
                print(f"Player 2 Wins with {self.board[self.p2_mancala_index]} stones")
            else:
                print(f"We have a tie at {self.board[self.p1_mancala_index]} stones")
            return True
        return False
    
    
    def prompt_move(self):
        """
        Prompt the current player for a move and validate it.
        """
        while True:
            self.display_board()
            
            move = input(f"Player {self.current_player}, select a pit (1-6): ")
            
            try:
                move = int(move)
            except ValueError:
                print("Please enter a valid number between 1 and 6.")
                continue
                
            if self.valid_move(move):
                self.play(move)
                break
                
            else:
                print("Invalid move. Try again.")
            

In [ ]:
playgame = Mancala()
while not playgame.winning_eval():
    playgame.prompt_move()
    
playgame.display_board()

P1               P2
     ____0____     
1 -> | 4 | 4 | <- 6
2 -> | 4 | 4 | <- 5
3 -> | 4 | 4 | <- 4
4 -> | 4 | 4 | <- 3
5 -> | 4 | 4 | <- 2
6 -> |_4_|_4_| <- 1
         0         
Turn: P1
